In [7]:
import numpy as np 
np.bool = np.bool_


In [1]:
import os

os.environ["AWS_REGION"] = "us-east-1" 
os.environ["AWS_ACCESS_KEY_ID"] = "admin"
os.environ["AWS_SECRET_ACCESS_KEY"] = "admin123"

In [2]:
import os
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkConf

findspark.init()

# Set Spark configuration
conf = SparkConf()
conf.set(
    "spark.jars.packages",
    "org.apache.hadoop:hadoop-aws:3.3.1,"
    "org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.0,"
    "org.projectnessie:nessie-spark-extensions-3.3_2.12:0.44.0,"
    "software.amazon.awssdk:bundle:2.17.178,"
    "software.amazon.awssdk:url-connection-client:2.17.178"
)
conf.set(
    "spark.sql.extensions",
    "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
)
conf.set("spark.sql.catalog.spark_catalog.type","hive")
conf.set("spark.sql.catalog.spark_catalog","org.apache.iceberg.spark.SparkSessionCatalog")
# Ensure Python <-> Java interactions are with PyArrow
conf.set("spark.sql.execution.pyarrow.enabled", "true")

# Set Hive support configuration
conf.set("spark.sql.catalogImplementation", "hive")

# Set the Hive metastore URI to point to the external CDP cluster
conf.set("hive.metastore.uris", "thrift://51.15.202.128:9083")



conf.set('spark.sql.catalog.iceberg', 'org.apache.iceberg.spark.SparkCatalog')
# set the location of the nessie server
conf.set('spark.sql.catalog.iceberg.uri', "http://212.47.236.230:19120/api/v1")
# default branch for Nessie catalog to work on
conf.set('spark.sql.catalog.iceberg.ref', 'main')
# use no authorization. Options are NONE AWS BASIC and aws implies running Nessie on a lambda
conf.set('spark.sql.catalog.iceberg.authentication.type', 'NONE')
# tell the nessie that its a Nessie catalog
conf.set('spark.sql.catalog.iceberg.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')

conf.set('spark.sql.catalog.iceberg.s3.endpoint', "http://212.47.236.230:9000")
# set the location for Nessie catalog to store data. Spark writes to this directory
conf.set('spark.sql.catalog.iceberg.warehouse', "s3a://warehouse/tablespace/external/hive/tpcds_10000_parquet.db")
conf.set('spark.sql.catalog.iceberg.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')


conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set("spark.hadoop.fs.s3a.access.key", "admin")
conf.set("spark.hadoop.fs.s3a.secret.key", "admin123")
conf.set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
conf.set("spark.sql.warehouse.dir", "s3a://warehouse/tablespace/external/hive/tpcds_10000_parquet.db/")
conf.set("spark.hadoop.fs.s3a.endpoint", "http://212.47.236.230:9000")  # Add your S3 endpoint here


In [3]:
spark = SparkSession.builder \
    .appName("CDP Cluster Connection") \
    .config(conf=conf) \
    .enableHiveSupport() \
    .getOrCreate()

print("Spark Running")

spark.sql("SHOW DATABASES").show()

spark.sql("USE default")

spark.sql("SHOW TABLES").show()


24/07/02 11:48:12 WARN Utils: Your hostname, MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 192.168.2.35 instead (on interface en0)
24/07/02 11:48:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/Ziad/anaconda3/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/Ziad/.ivy2/cache
The jars for the packages stored in: /Users/Ziad/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.3_2.12 added as a dependency
org.projectnessie#nessie-spark-extensions-3.3_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-40adcc08-197a-43f0-88df-2c789726257f;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in local-m2-cache
	found org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.3.0 in central
	found org.projectnessie#nessie-spark-extensions-3.3_2.12;0.44.0 in central
	found org.scala-lang#scala-reflect;2.12.17 in central
	found org.projectnessie#nessie-spark-extensions-grammar;0

24/07/02 11:48:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark Running
24/07/02 11:48:22 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
+--------------------+
|           namespace|
+--------------------+
|                 air|
|bi_modeling_test_...|
|             default|
|  default_mart_covid|
|   default_reference|
|default_staging_c...|
|             iceberg|
|  information_schema|
|          nemo_delta|
|                 sys|
| tpcds_10000_parquet|
|    tpcds_10000_text|
+--------------------+



+---------+-------------------+-----------+
|namespace|          tableName|isTemporary|
+---------+-------------------+-----------+
|  default|         land_yousr|      false|
|  default|         land_gab22|      false|
|  default|     land_gab_abstr|      false|
|  default|        land_getfac|      false|
|  default|        land_getvir|      false|
|  default|         land_solde|      false|
|  default|              yousr|      false|
|  default|     land_customers|      false|
|  default|               test|      false|
|  default|             ice_v2|      false|
|  default|external_callcenter|      false|
|  default|          inventory|      false|
|  default|       catalog_page|      false|
|  default|         inventoryy|      false|
|  default|        exinventory|      false|
|  default|      sales_target1|      false|
|  default|   raw_covid__cases|      false|
|  default|raw_covid__vaccines|      false|
|  default|       sample_table|      false|
+---------+-------------------+-

### Describe and fetch hive tables 

In [8]:
spark.sql("show tables IN spark_catalog.tpcds_10000_parquet").toPandas()

,namespace,tableName,isTemporary
0,tpcds_10000_parquet,inventory_final,False
1,tpcds_10000_parquet,customer,False
2,tpcds_10000_parquet,store_backup_,False
3,tpcds_10000_parquet,store,False
4,tpcds_10000_parquet,warehouse,False


In [9]:
spark.sql("Select * from spark_catalog.tpcds_10000_parquet.warehouse limit 10 ").toPandas()

,w_warehouse_sk,w_warehouse_id,w_warehouse_name,w_warehouse_sq_ft,w_street_number,w_street_name,w_street_type,w_suite_number,w_city,w_county,w_state,w_zip,w_country,w_gmt_offset
0,1,AAAAAAAABAAAAAAA,Conventional childr,977787.0,651,6th,Parkway,Suite 470,Riverside,Ziebach County,SD,59231,United States,-6.00
1,2,AAAAAAAACAAAAAAA,Important issues liv,138504.0,600,View First,Avenue,Suite P,Fairview,Walker County,AL,35709,United States,-6.00
2,3,AAAAAAAADAAAAAAA,Doors canno,294242.0,534,Ash Laurel,Dr.,Suite 0,Riverside,Ziebach County,SD,59231,United States,-6.00
3,4,AAAAAAAAEAAAAAAA,Bad cards must make.,621234.0,368,Wilson Elm,Drive,Suite 80,Fairview,Ziebach County,SD,55709,United States,-6.00
4,5,AAAAAAAAFAAAAAAA,None,NaN,None,None,None,None,Riverside,Walker County,AL,39231,United States,None
5,6,AAAAAAAAGAAAAAAA,"Local, mass universi",838797.0,957,Lincoln Adams,Dr.,Suite X,Riverside,Ziebach County,SD,59231,United States,-6.00
6,7,AAAAAAAAHAAAAAAA,Quite effectiv,662475.0,69,7th Sunset,Ct.,Suite 350,Pleasant Hill,Ziebach County,SD,53604,United States,-6.00
7,8,AAAAAAAAIAAAAAAA,"Plain, reluctant",514427.0,410,3rd,ST,Suite 370,Riverside,Ziebach County,SD,59231,United States,-6.00
8,9,AAAAAAAAJAAAAAAA,Rooms cook,73065.0,420,Spring,Wy,Suite I,Oak Grove,Ziebach County,SD,58370,United States,-6.00
9,10,AAAAAAAAKAAAAAAA,"National, comple",914242.0,269,Elm Madison,Street,Suite 90,Pleasant Hill,Ziebach County,SD,53604,United States,-6.00


In [23]:
spark.sql("ALTER TABLE spark_catalog.tpcds_10000_parquet.inventory_final PARTITION (inv_date_sk=2450850) set location 's3a://warehouse/tablespace/external/hive/tpcds_10000_parquet.db/inventory_final/inv_date_sk=2450850' ")






DataFrame[]

In [37]:
spark.sql(f" describe formatted spark_catalog.tpcds_10000_parquet.inventory_final PARTITION (inv_date_sk=2450815) ").toPandas()


,col_name,data_type,comment
0,inv_item_sk,int,None
1,inv_warehouse_sk,int,None
2,inv_quantity_on_hand,int,None
3,inv_date_sk,int,None
4,# Partition Information,,
5,# col_name,data_type,comment
6,inv_date_sk,int,None
7,,,
8,# Detailed Partition Information,,
9,Database,tpcds_10000_parquet,


In [28]:
base_command = """
ALTER TABLE spark_catalog.tpcds_10000_parquet.inventory_final 
PARTITION (inv_date_sk={inv_date_sk}) 
SET LOCATION 's3a://warehouse/tablespace/external/hive/tpcds_10000_parquet.db/inventory_final/inv_date_sk={inv_date_sk}'
"""
for inv_date_sk in inv_date_sk_values:
    command = base_command.format(inv_date_sk=inv_date_sk)
    spark.sql(command)
    print(f"Executed command for inv_date_sk={inv_date_sk}")

Executed command for inv_date_sk=2450815
Executed command for inv_date_sk=2450822
Executed command for inv_date_sk=2450829
Executed command for inv_date_sk=2450836
Executed command for inv_date_sk=2450843
Executed command for inv_date_sk=2450850
Executed command for inv_date_sk=2450857
Executed command for inv_date_sk=2450864
Executed command for inv_date_sk=2450871
Executed command for inv_date_sk=2450878
Executed command for inv_date_sk=2450885
Executed command for inv_date_sk=2450892
Executed command for inv_date_sk=2450899
Executed command for inv_date_sk=2450906
Executed command for inv_date_sk=2450913
Executed command for inv_date_sk=2450920
Executed command for inv_date_sk=2450927
Executed command for inv_date_sk=2450934
Executed command for inv_date_sk=2450941
Executed command for inv_date_sk=2450948
Executed command for inv_date_sk=2450955
Executed command for inv_date_sk=2450962
Executed command for inv_date_sk=2450969
Executed command for inv_date_sk=2450976
Executed command

Executed command for inv_date_sk=2452215
Executed command for inv_date_sk=2452222
Executed command for inv_date_sk=2452229
Executed command for inv_date_sk=2452236
Executed command for inv_date_sk=2452243
Executed command for inv_date_sk=2452250
Executed command for inv_date_sk=2452257
Executed command for inv_date_sk=2452264
Executed command for inv_date_sk=2452271
Executed command for inv_date_sk=2452278
Executed command for inv_date_sk=2452285
Executed command for inv_date_sk=2452292
Executed command for inv_date_sk=2452299
Executed command for inv_date_sk=2452306
Executed command for inv_date_sk=2452313
Executed command for inv_date_sk=2452320
Executed command for inv_date_sk=2452327
Executed command for inv_date_sk=2452334
Executed command for inv_date_sk=2452341
Executed command for inv_date_sk=2452348
Executed command for inv_date_sk=2452355
Executed command for inv_date_sk=2452362
Executed command for inv_date_sk=2452369
Executed command for inv_date_sk=2452376
Executed command

### Show the hive/iceberg catalogs

In [39]:
spark.sql("show catalogs").toPandas()

,catalog
0,iceberg
1,spark_catalog


In [6]:
spark.sql("show namespaces IN iceberg").toPandas()

,namespace
0,tpcds_10000_parquet


## Migrating the Hive Table to an Iceberg Table Without Restating the Data


## Using the migrate/add_files Procedures

In [10]:
spark.sql("Call spark_catalog.system.migrate(table => 'tpcds_10000_parquet.warehouse')").toPandas()

,migrated_files_count
0,1


In [11]:
spark.sql("CALL iceberg.system.add_files(table => 'iceberg.test.said',source_table => 'spark_catalog.tpcds_10000_parquet.inventory_final')").toPandas()

,added_files_count
0,348


In [35]:
df = spark.sql("desc formatted tpcds_10000_parquet.store").toPandas()
df

,col_name,data_type,comment
0,s_store_sk,int,
1,s_store_id,string,
2,s_rec_start_date,date,
3,s_rec_end_date,date,
4,s_closed_date_sk,int,
5,s_store_name,string,
6,s_number_employees,int,
7,s_floor_space,int,
8,s_hours,string,
9,s_manager,string,


In [36]:
f = df["col_name"] == "Table Properties"
df.where(f, inplace=True)
df.loc[44,"data_type"]

'[bucketing_version=2,current-snapshot-id=1925497414336440487,format=iceberg/parquet,format-version=1,last_modified_by=hive,last_modified_time=1719915093,migrated=true,numFilesErasureCoded=0,schema.name-mapping.default=[ {\n  "field-id" : 1,\n  "names" : [ "s_store_sk" ]\n}, {\n  "field-id" : 2,\n  "names" : [ "s_store_id" ]\n}, {\n  "field-id" : 3,\n  "names" : [ "s_rec_start_date" ]\n}, {\n  "field-id" : 4,\n  "names" : [ "s_rec_end_date" ]\n}, {\n  "field-id" : 5,\n  "names" : [ "s_closed_date_sk" ]\n}, {\n  "field-id" : 6,\n  "names" : [ "s_store_name" ]\n}, {\n  "field-id" : 7,\n  "names" : [ "s_number_employees" ]\n}, {\n  "field-id" : 8,\n  "names" : [ "s_floor_space" ]\n}, {\n  "field-id" : 9,\n  "names" : [ "s_hours" ]\n}, {\n  "field-id" : 10,\n  "names" : [ "s_manager" ]\n}, {\n  "field-id" : 11,\n  "names" : [ "s_market_id" ]\n}, {\n  "field-id" : 12,\n  "names" : [ "s_geography_class" ]\n}, {\n  "field-id" : 13,\n  "names" : [ "s_market_desc" ]\n}, {\n  "field-id" : 14,\n 

In [13]:
spark.sql('select * from tpcds_10000_parquet.store limit 3 ;').toPandas()

24/07/02 12:47:19 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,s_store_sk,s_store_id,s_rec_start_date,s_rec_end_date,s_closed_date_sk,s_store_name,s_number_employees,s_floor_space,s_hours,s_manager,...,s_street_name,s_street_type,s_suite_number,s_city,s_county,s_state,s_zip,s_country,s_gmt_offset,s_tax_precentage
0,1,AAAAAAAABAAAAAAA,1997-03-13,None,2451189.0,ought,245,5250760,8AM-4PM,William Ward,...,Spring,Wy,Suite 250,Pleasant Hill,Ziebach County,SD,53604,United States,-6.00,0.03
1,2,AAAAAAAACAAAAAAA,1997-03-13,2000-03-12,NaN,able,236,5285950,8AM-4PM,Scott Smith,...,Sycamore,Dr.,Suite 410,Midway,Ziebach County,SD,51904,United States,-6.00,0.03
2,3,AAAAAAAACAAAAAAA,2000-03-13,None,NaN,able,236,7557959,8AM-4PM,Scott Smith,...,Park Laurel,Road,Suite T,Midway,Williamson County,TN,31904,United States,-5.00,0.03


In [12]:
spark.sql("show tables in tpcds_10000_parquet ").toPandas()

,namespace,tableName,isTemporary
0,tpcds_10000_parquet,inventory_final,False
1,tpcds_10000_parquet,customer,False
2,tpcds_10000_parquet,store_backup_,False
3,tpcds_10000_parquet,store,False


In [20]:
spark.sql('show tables in iceberg').toPandas()

,namespace,tableName,isTemporary
0,,events,False
1,,names,False
2,nba,salaries,False
3,test,ara,False
4,test,customersice,False
5,test,fina,False
6,test,final,False
7,test,inventorisy,False
8,test,said,False
9,test,toto,False
